# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [29]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [30]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [31]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [32]:
chars[:100]

array([44, 79, 68, 42, 30, 67, 48, 59, 33, 72, 72, 72, 26, 68, 42, 42,  3,
       59, 77, 68,  8, 58, 24, 58, 67,  0, 59, 68, 48, 67, 59, 68, 24, 24,
       59, 68, 24, 58, 60, 67, 56, 59, 67, 34, 67, 48,  3, 59,  2, 69, 79,
       68, 42, 42,  3, 59, 77, 68,  8, 58, 24,  3, 59, 58,  0, 59,  2, 69,
       79, 68, 42, 42,  3, 59, 58, 69, 59, 58, 30,  0, 59, 10,  4, 69, 72,
        4, 68,  3, 53, 72, 72,  9, 34, 67, 48,  3, 30, 79, 58, 69], dtype=int32)

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[44, 79, 68, 42, 30, 67, 48, 59, 33, 72],
       [16, 69, 63, 59, 79, 67, 59,  8, 10, 34],
       [59, 61, 68, 30, 61, 79, 58, 69, 50, 59],
       [10, 30, 79, 67, 48, 59,  4, 10,  2, 24],
       [59, 30, 79, 67, 59, 24, 68, 69, 63, 62],
       [59, 32, 79, 48, 10,  2, 50, 79, 59, 24],
       [30, 59, 30, 10, 72, 63, 10, 53, 72, 72],
       [10, 59, 79, 67, 48,  0, 67, 24, 77, 37],
       [79, 68, 30, 59, 58,  0, 59, 30, 79, 67],
       [67, 48,  0, 67, 24, 77, 59, 68, 69, 63]], dtype=int32)

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [20]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')

    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    with tf.name_scope('RNN_layers'):
        # Build the RNN layers
        def build_BasicLSTMCell(lstm_size, keep_prob):
            lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
            return drop
        cell = tf.contrib.rnn.MultiRNNCell([build_BasicLSTMCell(lstm_size, keep_prob) for val in range(num_layers)])
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    with tf.name_scope("RNN_forward"):
        # Run the data through the RNN layers
        rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output    
    with tf.name_scope('sequence_shape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN putputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b', softmax_b)

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', preds)

    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost', cost)

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    merged = tf.summary.merge_all()
    
    # Export the nodes
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [21]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Write out the graph for TensorBoard

In [22]:
model = build_rnn(len(vocab),
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter('./logs/3', sess.graph)

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [23]:
!mkdir -p checkpoints/anna

In [24]:
epochs = 10
save_every_n = 100
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/2/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/2/test')
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state}
            summary, batch_loss, new_state, _ = sess.run([model.merged,model.cost, 
                                                          model.final_state, model.optimizer], feed_dict=feed)

            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            train_writer.add_summary(summary, iteration)
        
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps ):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    summary, batch_loss, new_state = sess.run([model.merged, model.cost, 
                                                               model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)
                test_writer.add_summary(summary, iteration)
                
                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/anna/i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/10  Iteration 1/1780 Training loss: 4.4200 3.5427 sec/batch
Epoch 1/10  Iteration 2/1780 Training loss: 4.3738 3.6093 sec/batch
Epoch 1/10  Iteration 3/1780 Training loss: 4.1854 3.4074 sec/batch
Epoch 1/10  Iteration 4/1780 Training loss: 4.3983 3.6861 sec/batch
Epoch 1/10  Iteration 5/1780 Training loss: 4.3457 4.1835 sec/batch
Epoch 1/10  Iteration 6/1780 Training loss: 4.2776 3.4802 sec/batch
Epoch 1/10  Iteration 7/1780 Training loss: 4.2079 4.3144 sec/batch
Epoch 1/10  Iteration 8/1780 Training loss: 4.1360 4.8322 sec/batch
Epoch 1/10  Iteration 9/1780 Training loss: 4.0631 4.7170 sec/batch
Epoch 1/10  Iteration 10/1780 Training loss: 4.0001 5.0246 sec/batch
Epoch 1/10  Iteration 11/1780 Training loss: 3.9438 4.7248 sec/batch
Epoch 1/10  Iteration 12/1780 Training loss: 3.8987 4.6540 sec/batch
Epoch 1/10  Iteration 13/1780 Training loss: 3.8578 4.3570 sec/batch
Epoch 1/10  Iteration 14/1780 Training loss: 3.8220 4.3667 sec/batch
Epoch 1/10  Iteration 15/1780 Training loss

Epoch 1/10  Iteration 119/1780 Training loss: 3.2254 3.8227 sec/batch
Epoch 1/10  Iteration 120/1780 Training loss: 3.2235 4.9693 sec/batch
Epoch 1/10  Iteration 121/1780 Training loss: 3.2219 5.0501 sec/batch
Epoch 1/10  Iteration 122/1780 Training loss: 3.2202 3.9961 sec/batch
Epoch 1/10  Iteration 123/1780 Training loss: 3.2183 3.8178 sec/batch
Epoch 1/10  Iteration 124/1780 Training loss: 3.2165 3.8345 sec/batch
Epoch 1/10  Iteration 125/1780 Training loss: 3.2146 3.8703 sec/batch
Epoch 1/10  Iteration 126/1780 Training loss: 3.2125 3.9015 sec/batch
Epoch 1/10  Iteration 127/1780 Training loss: 3.2105 3.8733 sec/batch
Epoch 1/10  Iteration 128/1780 Training loss: 3.2086 3.8445 sec/batch
Epoch 1/10  Iteration 129/1780 Training loss: 3.2065 3.9670 sec/batch
Epoch 1/10  Iteration 130/1780 Training loss: 3.2044 4.0543 sec/batch
Epoch 1/10  Iteration 131/1780 Training loss: 3.2022 3.9050 sec/batch
Epoch 1/10  Iteration 132/1780 Training loss: 3.1998 3.9409 sec/batch
Epoch 1/10  Iteratio

Epoch 2/10  Iteration 236/1780 Training loss: 2.4408 3.7921 sec/batch
Epoch 2/10  Iteration 237/1780 Training loss: 2.4394 3.7951 sec/batch
Epoch 2/10  Iteration 238/1780 Training loss: 2.4382 3.7903 sec/batch
Epoch 2/10  Iteration 239/1780 Training loss: 2.4375 3.7908 sec/batch
Epoch 2/10  Iteration 240/1780 Training loss: 2.4362 3.8147 sec/batch
Epoch 2/10  Iteration 241/1780 Training loss: 2.4352 3.7833 sec/batch
Epoch 2/10  Iteration 242/1780 Training loss: 2.4337 3.7931 sec/batch
Epoch 2/10  Iteration 243/1780 Training loss: 2.4321 3.7850 sec/batch
Epoch 2/10  Iteration 244/1780 Training loss: 2.4310 3.8014 sec/batch
Epoch 2/10  Iteration 245/1780 Training loss: 2.4296 3.8006 sec/batch
Epoch 2/10  Iteration 246/1780 Training loss: 2.4279 3.7939 sec/batch
Epoch 2/10  Iteration 247/1780 Training loss: 2.4261 3.8027 sec/batch
Epoch 2/10  Iteration 248/1780 Training loss: 2.4248 3.7906 sec/batch
Epoch 2/10  Iteration 249/1780 Training loss: 2.4236 3.7972 sec/batch
Epoch 2/10  Iteratio

Epoch 2/10  Iteration 353/1780 Training loss: 2.2962 5.3518 sec/batch
Epoch 2/10  Iteration 354/1780 Training loss: 2.2952 5.4143 sec/batch
Epoch 2/10  Iteration 355/1780 Training loss: 2.2941 5.3777 sec/batch
Epoch 2/10  Iteration 356/1780 Training loss: 2.2932 5.4279 sec/batch
Epoch 3/10  Iteration 357/1780 Training loss: 2.1736 5.4091 sec/batch
Epoch 3/10  Iteration 358/1780 Training loss: 2.1270 5.4482 sec/batch
Epoch 3/10  Iteration 359/1780 Training loss: 2.1155 5.3929 sec/batch
Epoch 3/10  Iteration 360/1780 Training loss: 2.1103 5.3984 sec/batch
Epoch 3/10  Iteration 361/1780 Training loss: 2.1081 5.4265 sec/batch
Epoch 3/10  Iteration 362/1780 Training loss: 2.1027 5.3785 sec/batch
Epoch 3/10  Iteration 363/1780 Training loss: 2.1031 5.4295 sec/batch
Epoch 3/10  Iteration 364/1780 Training loss: 2.1029 5.3972 sec/batch
Epoch 3/10  Iteration 365/1780 Training loss: 2.1052 5.3781 sec/batch
Epoch 3/10  Iteration 366/1780 Training loss: 2.1043 5.4089 sec/batch
Epoch 3/10  Iteratio

Epoch 3/10  Iteration 470/1780 Training loss: 2.0167 5.2321 sec/batch
Epoch 3/10  Iteration 471/1780 Training loss: 2.0159 5.1999 sec/batch
Epoch 3/10  Iteration 472/1780 Training loss: 2.0150 5.2752 sec/batch
Epoch 3/10  Iteration 473/1780 Training loss: 2.0142 5.2292 sec/batch
Epoch 3/10  Iteration 474/1780 Training loss: 2.0135 5.1965 sec/batch
Epoch 3/10  Iteration 475/1780 Training loss: 2.0129 5.2663 sec/batch
Epoch 3/10  Iteration 476/1780 Training loss: 2.0122 5.3606 sec/batch
Epoch 3/10  Iteration 477/1780 Training loss: 2.0117 5.5163 sec/batch
Epoch 3/10  Iteration 478/1780 Training loss: 2.0109 5.9272 sec/batch
Epoch 3/10  Iteration 479/1780 Training loss: 2.0100 5.4973 sec/batch
Epoch 3/10  Iteration 480/1780 Training loss: 2.0096 5.2069 sec/batch
Epoch 3/10  Iteration 481/1780 Training loss: 2.0089 5.2316 sec/batch
Epoch 3/10  Iteration 482/1780 Training loss: 2.0079 5.2441 sec/batch
Epoch 3/10  Iteration 483/1780 Training loss: 2.0074 5.1920 sec/batch
Epoch 3/10  Iteratio

...

Epoch 8/10  Iteration 1283/1780 Training loss: 1.4732 3.7824 sec/batch
Epoch 8/10  Iteration 1284/1780 Training loss: 1.4719 3.8012 sec/batch
Epoch 8/10  Iteration 1285/1780 Training loss: 1.4705 3.7848 sec/batch
Epoch 8/10  Iteration 1286/1780 Training loss: 1.4698 3.7853 sec/batch
Epoch 8/10  Iteration 1287/1780 Training loss: 1.4693 3.7937 sec/batch
Epoch 8/10  Iteration 1288/1780 Training loss: 1.4696 3.7758 sec/batch
Epoch 8/10  Iteration 1289/1780 Training loss: 1.4689 3.7935 sec/batch
Epoch 8/10  Iteration 1290/1780 Training loss: 1.4682 3.8745 sec/batch
Epoch 8/10  Iteration 1291/1780 Training loss: 1.4684 3.7870 sec/batch
Epoch 8/10  Iteration 1292/1780 Training loss: 1.4674 3.7943 sec/batch
Epoch 8/10  Iteration 1293/1780 Training loss: 1.4670 3.7935 sec/batch
Epoch 8/10  Iteration 1294/1780 Training loss: 1.4664 3.7869 sec/batch
Epoch 8/10  Iteration 1295/1780 Training loss: 1.4662 3.7866 sec/batch
Epoch 8/10  Iteration 1296/1780 Training loss: 1.4664 3.8035 sec/batch
Epoch 

Epoch 8/10  Iteration 1398/1780 Training loss: 1.4512 3.7922 sec/batch
Epoch 8/10  Iteration 1399/1780 Training loss: 1.4511 3.7913 sec/batch
Epoch 8/10  Iteration 1400/1780 Training loss: 1.4510 3.7915 sec/batch
Validation loss: 1.3233 Saving checkpoint!
Epoch 8/10  Iteration 1401/1780 Training loss: 1.4515 3.7291 sec/batch
Epoch 8/10  Iteration 1402/1780 Training loss: 1.4514 3.7986 sec/batch
Epoch 8/10  Iteration 1403/1780 Training loss: 1.4513 3.7958 sec/batch
Epoch 8/10  Iteration 1404/1780 Training loss: 1.4512 3.7984 sec/batch
Epoch 8/10  Iteration 1405/1780 Training loss: 1.4508 3.7949 sec/batch
Epoch 8/10  Iteration 1406/1780 Training loss: 1.4508 3.7960 sec/batch
Epoch 8/10  Iteration 1407/1780 Training loss: 1.4509 3.7969 sec/batch
Epoch 8/10  Iteration 1408/1780 Training loss: 1.4508 3.7937 sec/batch
Epoch 8/10  Iteration 1409/1780 Training loss: 1.4507 3.8064 sec/batch
Epoch 8/10  Iteration 1410/1780 Training loss: 1.4506 3.7915 sec/batch
Epoch 8/10  Iteration 1411/1780 Tr

Epoch 9/10  Iteration 1513/1780 Training loss: 1.4116 4.6834 sec/batch
Epoch 9/10  Iteration 1514/1780 Training loss: 1.4116 5.3768 sec/batch
Epoch 9/10  Iteration 1515/1780 Training loss: 1.4113 5.2227 sec/batch
Epoch 9/10  Iteration 1516/1780 Training loss: 1.4111 4.7540 sec/batch
Epoch 9/10  Iteration 1517/1780 Training loss: 1.4106 4.7751 sec/batch
Epoch 9/10  Iteration 1518/1780 Training loss: 1.4101 4.5470 sec/batch
Epoch 9/10  Iteration 1519/1780 Training loss: 1.4096 4.0631 sec/batch
Epoch 9/10  Iteration 1520/1780 Training loss: 1.4095 3.9526 sec/batch
Epoch 9/10  Iteration 1521/1780 Training loss: 1.4095 3.8920 sec/batch
Epoch 9/10  Iteration 1522/1780 Training loss: 1.4090 3.9649 sec/batch
Epoch 9/10  Iteration 1523/1780 Training loss: 1.4086 4.2309 sec/batch
Epoch 9/10  Iteration 1524/1780 Training loss: 1.4080 3.8640 sec/batch
Epoch 9/10  Iteration 1525/1780 Training loss: 1.4080 3.9292 sec/batch
Epoch 9/10  Iteration 1526/1780 Training loss: 1.4077 3.7797 sec/batch
Epoch 

Epoch 10/10  Iteration 1628/1780 Training loss: 1.3773 3.7602 sec/batch
Epoch 10/10  Iteration 1629/1780 Training loss: 1.3756 3.7750 sec/batch
Epoch 10/10  Iteration 1630/1780 Training loss: 1.3760 3.7696 sec/batch
Epoch 10/10  Iteration 1631/1780 Training loss: 1.3760 3.7653 sec/batch
Epoch 10/10  Iteration 1632/1780 Training loss: 1.3760 3.7680 sec/batch
Epoch 10/10  Iteration 1633/1780 Training loss: 1.3756 3.7748 sec/batch
Epoch 10/10  Iteration 1634/1780 Training loss: 1.3746 3.7686 sec/batch
Epoch 10/10  Iteration 1635/1780 Training loss: 1.3748 3.8759 sec/batch
Epoch 10/10  Iteration 1636/1780 Training loss: 1.3749 3.7865 sec/batch
Epoch 10/10  Iteration 1637/1780 Training loss: 1.3747 3.7704 sec/batch
Epoch 10/10  Iteration 1638/1780 Training loss: 1.3744 3.7622 sec/batch
Epoch 10/10  Iteration 1639/1780 Training loss: 1.3735 3.7780 sec/batch
Epoch 10/10  Iteration 1640/1780 Training loss: 1.3723 3.7800 sec/batch
Epoch 10/10  Iteration 1641/1780 Training loss: 1.3709 3.7652 se

Epoch 10/10  Iteration 1742/1780 Training loss: 1.3565 3.8140 sec/batch
Epoch 10/10  Iteration 1743/1780 Training loss: 1.3568 3.8028 sec/batch
Epoch 10/10  Iteration 1744/1780 Training loss: 1.3568 3.8136 sec/batch
Epoch 10/10  Iteration 1745/1780 Training loss: 1.3567 3.8644 sec/batch
Epoch 10/10  Iteration 1746/1780 Training loss: 1.3569 3.9156 sec/batch
Epoch 10/10  Iteration 1747/1780 Training loss: 1.3567 3.7955 sec/batch
Epoch 10/10  Iteration 1748/1780 Training loss: 1.3568 3.8007 sec/batch
Epoch 10/10  Iteration 1749/1780 Training loss: 1.3568 3.8150 sec/batch
Epoch 10/10  Iteration 1750/1780 Training loss: 1.3570 3.7830 sec/batch
Epoch 10/10  Iteration 1751/1780 Training loss: 1.3571 3.7994 sec/batch
Epoch 10/10  Iteration 1752/1780 Training loss: 1.3568 3.8511 sec/batch
Epoch 10/10  Iteration 1753/1780 Training loss: 1.3566 3.8527 sec/batch
Epoch 10/10  Iteration 1754/1780 Training loss: 1.3564 3.8005 sec/batch
Epoch 10/10  Iteration 1755/1780 Training loss: 1.3564 3.8157 se

In [25]:
tf.train.get_checkpoint_state('checkpoints/anna')

model_checkpoint_path: "checkpoints/anna/i1780_l512_1.242.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i100_l512_3.033.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i200_l512_2.381.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i300_l512_2.117.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i400_l512_1.933.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i500_l512_1.811.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i600_l512_1.715.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i700_l512_1.620.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i800_l512_1.551.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i900_l512_1.496.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1000_l512_1.450.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1100_l512_1.414.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1200_l512_1.377.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1300_l512_1.344.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [26]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [27]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [28]:
checkpoint = "checkpoints/anna/i3560_l512_1.122.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i3560_l512_1.122.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/anna/i3560_l512_1.122.ckpt
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]

Caused by op 'save/RestoreV2_2', defined at:
  File "/home/maheshubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/maheshubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/maheshubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/maheshubuntu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/maheshubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in st

In [ ]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [24]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i600_l512_1.750.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/anna/i600_l512_1.750.ckpt
	 [[Node: save/RestoreV2_19 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_19/tensor_names, save/RestoreV2_19/shape_and_slices)]]
	 [[Node: save/RestoreV2_17/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_180_save/RestoreV2_17", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_19', defined at:
  File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_n

In [47]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farrat, his felt has at it.

"When the pose ther hor exceed
to his sheant was," weat a sime of his sounsed. The coment and the facily that which had began terede a marilicaly whice whether the pose of his hand, at she was alligated herself the same on she had to
taiking to his forthing and streath how to hand
began in a lang at some at it, this he cholded not set all her. "Wo love that is setthing. Him anstering as seen that."

"Yes in the man that say the mare a crances is it?" said Sergazy Ivancatching. "You doon think were somether is ifficult of a mone of
though the most at the countes that the
mean on the come to say the most, to
his feesing of
a man she, whilo he
sained and well, that he would still at to said. He wind at his for the sore in the most
of hoss and almoved to see him. They have betine the sumper into at he his stire, and what he was that at the so steate of the
sound, and shin should have a geest of shall feet on the conderation to she had been at that imporsing the